In [1]:
from knn_no_spec_exp import *

db = pd.read_csv('data/lc_db_processed.csv').drop(columns = ['Unnamed: 0','smiles', 'test_cas'])

db.drop(columns = ['obs_duration_mean', 'conc1_type', 'exposure_type', 'control_type',
       'media_type', 'application_freq_unit','species', 'class',
       'tax_order', 'family', 'genus'], inplace = True)


# K-NN Senza Pubchem

In [2]:
X = db.drop(columns = ['conc1_mean','pubchem2d']).copy()
y = np.where(db['conc1_mean'].values>1,1,0)
for nc in X.columns:
    minmax = MinMaxScaler()
    minmax.fit(X[[nc]])
    X[[nc]] = minmax.transform(X[[nc]])

In [4]:
knn_metrics(X, y, k = 1, ls = 100)

Accuracy: 	 0.8193002257336344, se: 0.004021449605264086
RMSE: 	 	 0.4250084966756271, se: 0.004745243930825608
Sensitivity: 	 0.8611191664221575, se: 0.014654054882054084
Precision: 	 0.8504715349975409, se: 0.010236996794484842
Specificity: 	 0.7511932048646786, se: 0.02316219280810622


# CV

In [5]:
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.33, random_state = 42)
del _

best_k, best_leaf = cv_knn(X_train, y_train)

New best params found! k:1, leaf:10, acc:  0.8106047843665767, st.error:  0.006235951252291087,rmse: 0.4350142705957616, st.error:  0.007252586009157814
New best params found! k:1, leaf:20, acc:  0.816669474393531, st.error:  0.005150495415017391,rmse: 0.4280441901360305, st.error:  0.006019327129027978
New best params found! k:1, leaf:50, acc:  0.8184804582210242, st.error:  0.003711619515499288,rmse: 0.42598415116568766, st.error:  0.004360609079182598
New best params found! k:3, leaf:10, acc:  0.8319996630727763, st.error:  0.0031058396366021378,rmse: 0.4098260459453583, st.error:  0.003783692729630788
New best params found! k:3, leaf:20, acc:  0.8349477762803235, st.error:  0.0023184157469916954,rmse: 0.4062360751915161, st.error:  0.002856275702693064
New best params found! k:3, leaf:70, acc:  0.8352004716981132, st.error:  0.004036906880068498,rmse: 0.40586363056959374, st.error:  0.004974658539541558
New best params found! k:3, leaf:90, acc:  0.8410966981132075, st.error:  0.004

# Final Model

In [6]:
best_k = 7
best_leaf = 70

knn_metrics(X,y, best_k, best_leaf)

Accuracy: 	 0.8442720090293454, se: 0.0013963705453339445
RMSE: 	 	 0.3946120242235179, se: 0.0017645876640539082
Sensitivity: 	 0.8703123998631606, se: 0.010271281991218628
Precision: 	 0.8771195903294841, se: 0.006631900990046107
Specificity: 	 0.802177578139511, se: 0.013976648953808353


# K-NN con Pubchem2d

Pubchem2d have been transformed from string format to dataframe and concatenated to the other chemical descriptors.

The metric is the Euclidean.

In [2]:
X = db.drop(columns = ['conc1_mean']).copy()
y = np.where(db['conc1_mean'].values>1,1,0)
for nc in X.drop(columns = 'pubchem2d').columns:
    minmax = MinMaxScaler()
    minmax.fit(X[[nc]])
    X[[nc]] = minmax.transform(X[[nc]])
    
pub = pd.DataFrame(pd.DataFrame(X['pubchem2d'].values).apply(
        lambda x: x.str.replace('', ' ').str.strip().str.split(' '),axis = 1)[0].to_list())
X_new = X.drop(columns = 'pubchem2d').copy()
X_new = pd.concat([X_new, pub], axis = 1)

In [9]:
knn_metrics(X_new, y, k = 1, ls = 100)

Accuracy: 	 0.8106659142212189, se: 0.005050606156615719
RMSE: 	 	 0.43500891826809324, se: 0.0058116780710464125
Sensitivity: 	 0.8509902130269467, se: 0.006373262374104756
Precision: 	 0.8435329489706903, se: 0.005533459610200883
Specificity: 	 0.7455372527240937, se: 0.008872143225625543


# CV

In [4]:
X_train, _, y_train, _ = train_test_split(X_new, y, test_size=0.33, random_state = 42)
del _

best_k, best_leaf = cv_knn(X_train, y_train)

New best params found! k:1, leaf:10, acc:  0.8104784366576818, st.error:  0.001394284480900165,rmse: 0.43533189607118883, st.error:  0.001602457007402721
New best params found! k:1, leaf:40, acc:  0.8170064016172507, st.error:  0.005837523444635115,rmse: 0.42761043694025763, st.error:  0.006901994941350545
New best params found! k:1, leaf:50, acc:  0.81776448787062, st.error:  0.003793471751490935,rmse: 0.42682214278513836, st.error:  0.004410992242228689
New best params found! k:1, leaf:100, acc:  0.8225235849056604, st.error:  0.0035031385979189464,rmse: 0.42121775656020455, st.error:  0.0041639985882120974
New best params found! k:3, leaf:10, acc:  0.832336590296496, st.error:  0.0018914682928643704,rmse: 0.40944755140739925, st.error:  0.0023174950549518006
New best params found! k:3, leaf:20, acc:  0.8354531671159029, st.error:  0.0022363769197166136,rmse: 0.4056158644444108, st.error:  0.0027448980011671844
New best params found! k:3, leaf:30, acc:  0.8361691374663072, st.error: 

# Final Model

In [4]:
best_k = 7
best_leaf = 30
knn_metrics(X_new, y, k = best_k, ls = best_leaf)

Accuracy: 	 0.8489559819413093, se: 0.0031981282833046546
RMSE: 	 	 0.38857849913452525, se: 0.004113719689464835
Sensitivity: 	 0.8778587936403173, se: 0.007399104627168274
Precision: 	 0.8799367292683733, se: 0.0030024630275927115
Specificity: 	 0.8010261231676522, se: 0.006667235597186932


# Random Forest

In [2]:
X = db.drop(columns = ['conc1_mean']).copy()
y = np.where(db['conc1_mean'].values>1,1,0)

for nc in X.drop(columns = 'pubchem2d').columns:
    minmax = MinMaxScaler()
    minmax.fit(X[[nc]])
    X[[nc]] = minmax.transform(X[[nc]])
    
pub = pd.DataFrame(pd.DataFrame(X['pubchem2d'].values).apply(
        lambda x: x.str.replace('', ' ').str.strip().str.split(' '),axis = 1)[0].to_list())
X_new = X.drop(columns = 'pubchem2d').copy()
X_new = pd.concat([X_new, pub], axis = 1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.33, stratify = y, random_state = 42)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print('''Accuracy: {},
RMSE: \t {},
Sensitivity: {},
Precision: {},
Specifity: {}
'''.format(accuracy_score(y_test, y_pred), sqrt(mean_squared_error(y_test, y_pred)), recall_score(y_test, y_pred),
          precision_score(y_test, y_pred), tn/(tn+fp)))

Accuracy: 0.8604966139954854,
RMSE: 	 0.3735015207526131,
Sensitivity: 0.8864462209302325,
Precision: 0.8887067395264117,
Specifity: 0.817938021454112

